In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---
# 0. データセットの読み込み

In [2]:
HR_DATASET_PATH = 'datasets/HR_comma_sep.csv'
hr_df = pd.read_csv(HR_DATASET_PATH)
hr_df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
hr_df.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'sales', 'salary'],
      dtype='object')

In [4]:
hr_df.shape

(14999, 10)

In [5]:
hr_df.salary.unique()

array(['low', 'medium', 'high'], dtype=object)

In [6]:
# salary(給与水準)をダミー変数へ置換する
hr_df.salary.replace({'low': 1, 'medium': 2, 'high': 3}, inplace=True)
# salesをダミー変数へ
hr_df = pd.get_dummies(hr_df, columns=['sales'])

In [8]:
hr_df.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'salary', 'sales_IT', 'sales_RandD',
       'sales_accounting', 'sales_hr', 'sales_management', 'sales_marketing',
       'sales_product_mng', 'sales_sales', 'sales_support', 'sales_technical'],
      dtype='object')

---
# ロジスティック回帰モデル
$$
{p}_{i} = logistic\left({z}_{i}\right) = \frac { 1 }{ 1+\exp\left(-{z}_{i}\right) } \quad 0 \le {p}_{i} \le 1\\
\log {\frac {p_{i}}{1-p_{i}}} = z_{i}\\
{z}_{i} = {\beta}_{1}+{\beta}_{2}{x}_{1i}+\cdots+{\beta}_{n}{x}_{ni}
$$

## オッズ比 - リスクの大きさ
$$
生存する確率/生存しない確率\\
\frac {p\left(x\right)}{1-p\left(x\right)} = \exp { \left\{{\beta}_{0} + {\beta}_{1}{x}_{1} + {\beta}_{2}{x}_{2} + \cdots + {\beta}_{n}{x}_{n}\right\} }
$$

 - ${x}_{i}$が限界的に1単位増加した場合
$$
\frac {p_{1}}{1-p_{1}} = \exp {\left\{ {\beta}_{0}+{\beta}_{1}{x}_{1}+ \cdots + {\beta}_{i}\left({x}_{i}+1\right) + \cdots + {\beta}_{n}{x}_{n}\right\}}\\
\frac {p_{0}}{1-p_{0}} = \exp {\left\{ {\beta}_{0}+{\beta}_{1}{x}_{1}+ \cdots + {\beta}_{i}{x}_{i} + \cdots + {\beta}_{n}{x}_{n}\right\}}\\
$$
その比をとると
$$
\frac {p_{1}}{1-p_{1}} \div \frac {p_{0}}{1-p_{0}} = \frac { \exp {\left\{ {\beta}_{0}+{\beta}_{1}{x}_{1}+ \cdots + {\beta}_{i}\left({x}_{i}+1\right) + \cdots + {\beta}_{n}{x}_{n}\right\}} }{ \exp {\left\{ {\beta}_{0}+{\beta}_{1}{x}_{1}+ \cdots + {\beta}_{i}{x}_{i} + \cdots + {\beta}_{n}{x}_{n}\right\}} }\\
\quad =\frac { exp\left\{ { \beta  }_{ i }\left( { x }_{ i }+1 \right)  \right\}  }{ exp\left\{ { \beta  }_{ i }{ x }_{ i } \right\}  } = \frac { exp\left\{ { \beta  }_{ i }{ x }_{ i } \right\} exp\left\{ { \beta  }_{ i } \right\}  }{ exp\left\{ { \beta  }_{ i }{ x }_{ i } \right\}  }  = exp\left\{ { \beta  }_{ i } \right\} 
$$
変数${x}_{i}$が1単位増加した場合、そうでないものに比べて$exp\left\{ { \beta  }_{ i } \right\}$倍の発症オッズを有すると推測できる。つまり、**1単位増加するに伴って生じるオッズの増大を意味する**。連続変数の場合、1単位の変化は小さすぎるので、10単位、50単位と適当に大きくして表現する。  


 - ${x}_{i}$がY単位増加した場合
$$
\frac { p_{ 1 } }{ 1-p_{ 1 } } \div \frac { p_{ 0 } }{ 1-p_{ 0 } } =\frac { { exp\left\{ { \beta  }_{ 0 }+{ \beta  }_{ 1 }{ x }_{ 1 }+\cdots +{ \beta  }_{ i }\left( { x }_{ i }+Y \right) +\cdots +{ \beta  }_{ n }{ x }_{ n } \right\}  } }{ { exp\left\{ { \beta  }_{ 0 }+{ \beta  }_{ 1 }{ x }_{ 1 }+\cdots +{ \beta  }_{ i }{ x }_{ i }+\cdots +{ \beta  }_{ n }{ x }_{ n } \right\}  } } \\ \quad =\frac { exp\left\{ { \beta  }_{ i }\left( { x }_{ i }+Y \right)  \right\}  }{ exp\left\{ { \beta  }_{ i }{ x }_{ i } \right\}  } =\frac { exp\left\{ { \beta  }_{ i }{ x }_{ i } \right\} exp\left\{ Y{ \beta  }_{ i } \right\}  }{ exp\left\{ { \beta  }_{ i }{ x }_{ i } \right\}  } ={ \left( exp\left\{ { \beta  }_{ i } \right\}  \right)  }^{ Y }
$$

In [9]:
# ロジスティック回帰の実行
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

use_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'Work_accident',
    'promotion_last_5years',
    'salary',
    'sales_IT',
    'sales_RandD',
    'sales_accounting',
    'sales_hr',
    'sales_management', 
    'sales_marketing',
    'sales_product_mng',
    'sales_sales',
    'sales_support', 
    'sales_technical'
]


# 離職者数:在籍者数 = 1:1に直す
X1 = hr_df[hr_df.left == 1][use_cols]
X0 = hr_df[hr_df.left == 0][use_cols].sample(len(X1))
X = pd.concat([X1, X0])
Y = hr_df.loc[X.index, 'left']


# 交差検証(ホールドアウト法)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, train_size=0.7, random_state=0)

logistic = LogisticRegression()
logistic.fit(x_train, y_train)

print('学習データ数: %s' % x_train.shape[0])
print('学習データのうち離職した人数: %s' % y_train[y_train == 1].shape[0])
print('検証データ数: %s' % x_test.shape[0])
print('検証データのうち離職した人数: %s' % y_test[y_test == 1].shape[0])

y_pred = logistic.predict(x_test)

print('---モデルの評価---')
print('正確度: %s' % accuracy_score(y_test, y_pred))
print('適合率: %s' % precision_score(y_test, y_pred))
print('再現率: %s' % recall_score(y_test, y_pred))
print('F値: %s' % f1_score(y_test, y_pred))

print('分割表')
confusion_df = pd.DataFrame(confusion_matrix(y_test, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
confusion_df

/Users/taiyou/.pyenv/versions/anaconda3-2.3.0/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


学習データ数: 4999
学習データのうち離職した人数: 2510
検証データ数: 2143
検証データのうち離職した人数: 1061
---モデルの評価---
正確度: 0.76574895007
適合率: 0.753399818676
再現率: 0.783223374175
F値: 0.768022181146
分割表


,在籍すると予測,離職すると予測
在籍者,810,272
離職者,230,831


In [10]:
# モデルの詳細
coeff_df = DataFrame([X.columns, logistic.coef_[0]]).T
coeff_df.rename(columns={0: '変数', 1: '回帰係数'}, inplace=True)

# 切片の追加
intercept_df = DataFrame({'変数': ['intercept_'], '回帰係数': [logistic.intercept_[0]]}, columns=['変数', '回帰係数'])
parameters_df = pd.concat([coeff_df, intercept_df])

# オッズ比の追加
Nth_power_of_odds  = {
    'satisfaction_level': 0.1,
    'last_evaluation': 0.1,
    'number_project': 1,
    'average_montly_hours': 8,
    'time_spend_company': 2,
    'Work_accident': 1,
    'promotion_last_5years': 1,
    'salary': 1
}

def add_odds(row):
    variable, beta = row
    if variable in Nth_power_of_odds:
        n_th = Nth_power_of_odds[variable]
    else:
        n_th = 1
    return np.exp(beta)**n_th

parameters_df['限界効果'] = parameters_df[['変数', '回帰係数']].apply(add_odds, axis=1)
parameters_df

,変数,回帰係数,限界効果
0,satisfaction_level,-4.636527,0.628982
1,last_evaluation,1.418542,1.152409
2,number_project,-0.4596459,0.631507
3,average_montly_hours,0.006091948,1.049943
4,time_spend_company,0.459432,2.506441
5,Work_accident,-1.342742,0.261129
6,promotion_last_5years,-0.9246012,0.396690
7,salary,-0.6899357,0.501608
8,sales_IT,-0.1680172,0.845339
9,sales_RandD,-0.3496682,0.704922


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, r2_score


use_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'Work_accident',
    'promotion_last_5years',
    'salary',
    'sales_IT',
    'sales_RandD',
    'sales_accounting',
    'sales_hr',
    'sales_management', 
    'sales_marketing',
    'sales_product_mng',
    'sales_sales',
    'sales_support', 
    'sales_technical'
]


# 離職者数:在籍者数 = 1:1に直す
X1 = hr_df[hr_df.left == 1][use_cols]
Y1 = hr_df[hr_df.left == 1]['left']
X0 = hr_df[hr_df.left == 0][use_cols + ['left']].sample(len(X1), random_state=0)
Y0 = X0.left
X0.drop('left', axis=1, inplace=True)
X = pd.concat([X1, X0], ignore_index=True)
Y = pd.concat([Y1, Y0], ignore_index=True)

# 交差検証(k-分割交差検証法)
CV = 10
y_pred = cross_val_predict(LogisticRegression(), X, Y, cv=CV)

print('正確度: %s' % accuracy_score(Y, y_pred))
print('適合率: %s' % precision_score(Y, y_pred))
print('再現率: %s' % recall_score(Y, y_pred))
print('F値: %s' % f1_score(Y, y_pred))
print('分割表')
confusion_df = pd.DataFrame(confusion_matrix(Y, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
confusion_df

正確度: 0.778353402408
適合率: 0.763940520446
再現率: 0.805656678801
F値: 0.784244241516
分割表


,在籍すると予測,離職すると予測
在籍者,2682,889
離職者,694,2877


In [11]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


use_cols = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'Work_accident',
    'promotion_last_5years'
]


# 離職者数:在籍者数 = 1:1に直す
X1 = hr_df[hr_df.left == 1][use_cols]
Y1 = hr_df[hr_df.left == 1]['left']
X0 = hr_df[hr_df.left == 0][use_cols + ['left']].sample(len(X1), random_state=0)
Y0 = X0.left
X0.drop('left', axis=1, inplace=True)
X = pd.concat([X1, X0], ignore_index=True)
Y = pd.concat([Y1, Y0], ignore_index=True)

# 交差検証(k-分割交差検証法)
CV = 10

logistic_cv = LogisticRegressionCV(scoring='recall', random_state=0, cv=CV)
logistic_cv.fit(X, Y)

print(logistic_cv.scores_[1])

y_pred = logistic_cv.predict(X)

print('正確度: %s' % accuracy_score(Y, y_pred))
print('適合率: %s' % precision_score(Y, y_pred))
print('再現率: %s' % recall_score(Y, y_pred))
print('F値: %s' % f1_score(Y, y_pred))
print('分割表')
confusion_df = pd.DataFrame(confusion_matrix(Y, y_pred), index=['在籍者', '離職者'], columns=['在籍すると予測', '離職すると予測'])
confusion_df

[[ 0.51955307  0.52234637  0.93575419  0.83798883  0.81843575  0.81843575
   0.81843575  0.81843575  0.81843575  0.81843575]
 [ 0.56582633  0.57142857  0.89915966  0.77310924  0.76190476  0.75910364
   0.767507    0.767507    0.767507    0.767507  ]
 [ 0.54901961  0.54621849  0.88235294  0.78151261  0.75910364  0.75630252
   0.75630252  0.75630252  0.75630252  0.75630252]
 [ 0.56862745  0.56022409  0.91036415  0.80952381  0.78991597  0.78711485
   0.78711485  0.78711485  0.78711485  0.78711485]
 [ 0.53221289  0.53501401  0.89635854  0.78991597  0.767507    0.76190476
   0.767507    0.767507    0.767507    0.767507  ]
 [ 0.49019608  0.49579832  0.91596639  0.82633053  0.82072829  0.81232493
   0.81232493  0.81232493  0.81232493  0.81232493]
 [ 0.56302521  0.56862745  0.92436975  0.79551821  0.79551821  0.79551821
   0.79551821  0.79551821  0.79551821  0.79551821]
 [ 0.51540616  0.51820728  0.91596639  0.81232493  0.78991597  0.79831933
   0.79831933  0.79831933  0.79831933  0.79831933]


,在籍すると予測,離職すると予測
在籍者,2682,889
離職者,340,3231


$$
{p} = \frac {1}{1+\exp\left(-z\right)}\\
z = 0.5897 -4.458{x}_{0} + 1.165{x}_{1} - 0.412{x}_{2} + 0.0045{x}_{3} + 0.488{x}_{4} -1.396{x}_{5} -1.518{x}_{6} -1.518{x}_{7}
$$

---
# 推定

## 信頼区間
### a. Wald信頼区間
説明変数$x_{k} \quad \left(k=1,\cdots, r\right)$の係数${\beta}_{k}$の95%信頼区間は、${\beta}_{k}$の漸近正規性を利用して
$$
\hat {{\beta}_{k}} \pm 1.96 \cdot s.e.\left(\hat {{\beta}_{k}}\right)
$$
で計算できる。ここで、$s.e.\left(\hat {{\beta}_{k}}\right)$は共分散行列$\sum {\left(\hat {\beta}\right)}$の$k+1$番目の対角成分の平方根である。$100(1-\alpha)$%信頼区間を求めるのであらば、係数1.96を標準正規分布の上側$100\left(\alpha/2\right)$%点$Z\left(\alpha/2\right)$に置き換えればよい。

## オッズ比の推定と解釈
### カテゴリー変数の場合
オッズ比の推定を考える。まず、$j$番目の説明変数が**カテゴリー変数**で、その第$s$カテゴリーの第$t$カテゴリーに対するオッズ比
$$
{ \varphi  }_{ st }={\frac {P_{s}}{1-P_{s}}}/{\frac {P_{t}}{1-P_{t}}}
$$
を考える。その対数は、
$$
\log {{ \varphi  }_{ st }}=\log { {\frac {P_{s}}{1-P_{s}}}/{\frac {P_{t}}{1-P_{t}}} }=\log {\frac {P_{s}}{1-P_{s}}} - \log {\frac {P_{t}}{1-P_{t}}}
$$
となる。**もし、他の説明変数が同じ値をもてば**上式は
$$
\log {{ \varphi  }_{ st }} = {\beta}_{js} - {\beta}_{jt}
$$
となる。つまり、
$$
\hat {{ \varphi  }_{ st }} = \exp {\left(\hat {{\beta}_{js}} - \hat {{\beta}_{jt}}\right)}
$$
で推定される。これがロジスティックモデルによる調整オッズ比(adjusted odds ratio)である。**あてはめたモデルの中で**統計的に有意であるかどうかの検定
<div style="text-align:center">
帰無仮説 $H_{0}: { \varphi  }_{ st }=1$  
対立仮説 $H_{1}: { \varphi  }_{ st }\neq1$
</div>
は次のWald検定により簡単に行うことができる。
$$
{ \chi  }^{ 2 }={ \left( \frac { \hat { { \beta  }_{ js } } -\hat { { \beta  }_{ jt } }  }{ s.e.\left( \hat { { \beta  }_{ js } } -\hat { { \beta  }_{ jt } }  \right)  }  \right)  }^{ 2 }\sim 自由度１の{ \chi  }^{ 2 }分布
$$
次にオッズ比${ \varphi  }_{ st }$の95%信頼区間を求める。Wald検定による差$\left( \hat { { \beta  }_{ js } } -\hat { { \beta  }_{ jt } }  \right)$の95%信頼区間は
$$
\left( \hat { { \beta  }_{ js } } -\hat { { \beta  }_{ jt } }  \right) \pm 1.96s.e.\left( \hat { { \beta  }_{ js } } -\hat { { \beta  }_{ jt } }  \right)
$$
で計算される。



### 連続変数の場合
説明変数が連続変数の場合を考える。$j$番目の説明変数の係数${\beta}_{j}$は変数${x}_{j}$の比例定数であり、「１単位当たりの増加」に対する対数オッズの増加分を意味している。つまり、単位当たりのオッズ比${\varphi}_{j}$は、
$$
\hat {{\varphi}_{j}} = \exp {\left({\beta}_{j}\right)}
$$
で推定される。あてはめたモデルの中でその有意性の仮説検定
<div style="text-align:center">
帰無仮説 $H_{0}: { \varphi  }_{ j }=1$  
対立仮説 $H_{1}: { \varphi  }_{ j }\neq1$
</div>
に対するWald検定は次式で与えられる。
$$
{ \chi  }^{ 2 }={ \left( \frac { \hat { { \beta  }_{ j } }  }{ s.e.\left( \hat { { \beta  }_{ j } }  \right)  }  \right)  }^{ 2 }\sim 自由度１の{ \chi  }^{ 2 }分布
$$

---
# モデルの適合度

モデルを評価するプロセス

 - **モデルの適合度(goodness-of-fit)の評価**[(擬似)決定係数、予測精度]：モデルがどの程度データに適合しているかを、データと推定値との差で定義される残差などを用いて評価することであり、程良く小さければモデルが適合していると判断する。
 - **モデル(変数)の有意性(significance of model)**[有意性検定、AIC]：モデルが適合している、いないにかかわらず適用したモデル(変数)が全く意味のないものであるか、少しは有意なものであるか、つまり、次の仮説検定に対応する
     - $H_{0}: {\beta}_{k} = 0$（ある変数の意味がない）
     - $H_{1}: {\beta}_{k} \neq 0$（ある変数の意味がある）
     - $H_{0}: {\beta}_{1}={\beta}_{2}=\cdots={\beta}_{r}=0$(モデルは意味がない)
     - $H_{1}: {\beta}_{1}={\beta}_{2}=\cdots={\beta}_{r}=0ではない$(モデルは意味がある)


### 尤度比指数(LRI): マックファーデンの擬似決定係数
多重回帰モデルで当てはまりの尺度として使われていた$R^{2}$や${\bar {R}}^{2}$は、あくまでも線形回帰モデルの場合に妥当な当てはまりの尺度であり、非線形モデルであるロジッット/プロビット・モデルに用いるのは適切ではない。このような場合、当てはまりの参考指標となるものの一つとして、最大化された対数尤度の値がある。この最大化された対数尤度の値を$\log {\hat {L}}$と表記する。  
　また、対数尤度と関連して、**尤度比指数(LRI)**というものもしばしば参照される。今、説明変数が定数のみで${x}_{1i}, \cdots, {x}_{ki}$を用いないでモデルを推定した場合の最大化された対数尤度の値を$\log {\hat {{L}_{0}}}$とすると、尤度比指数は以下で定義される。
$$
{{R}_{Mc}}^{2} \equiv LRI = 1 - \frac { \log { \hat { L }  }  }{ \log { \hat { { L }_{ 0 } }  }  } \\
0　\le LRI < 1
$$

 - モデルの定数以外の説明変数が全く説明力を持たない場合には、$LRI=0$となる
 - 説明変数を増やせば、$\log { \hat { L }  }$は通常は減少することはなく大きくなるので、LRIも通常の$R^{2}$と同じく増加する。
 - McFaddenの擬似決定係数は, 一般に$R^{2}$と比べて小さくなりがちであることが指摘される.

### Estrellaの擬似決定係数
> Estrella は, 決定係数が間隔尺度であることを重視した. 決定係数において, ゼロと1の意味するところは明らかだが, ゼロと1の間はどうだろうか. 直感的には, 0.5 なら半分あてはまり, 0.75なら 4分の3ほど当てはまってる, と考えるだろう. しかし, 先に上げた McFadden, Cox and Snell のいずれも, そうはならない. McFadden は決定係数を常に下回るし, Cox and Snell は上限が1より小さくなる. よって, これらの擬似決定係数が 0.5 の値をとっても, 実際には半分程度の当てはまりを意味しない. 一方で, 間隔尺度とは, 「対象物の大きさに対して等間隔に数字を与えるものさし」のことである. 間隔尺度であればより直感的に, どの程度当てはまりが良いかを知ることができる. Estrella の擬似決定係数は, 間隔尺度の性質を持つため, 間隔指数とも呼ばれる.

$$
{ { R }_{ E } }^{ 2 }={ \left[ \frac { \log { \hat { { L }_{ 0 } }  }  }{ \log { \hat { { L }_{ 1 } }  }  }  \right]  }^{ \cfrac { 2 }{ N } \log { \hat { { L }_{ 0 } }  }  }
$$

### モデルの適合率(hit rate)
> $i$番目の個人が選択肢$j$(たとえば1)を実際に選択していたとする。モデルの推計から得られた$j$番目の選択確率$\hat {p_{j}}$(たとえば1)が他の選択確率$\hat {p_{i}}$(たとえばゼロ)を上回っていたら、モデルが正しく選択を予測したと考えるのは自然である。    

1と0の選択を正しく予測された数を${T}_{1}$、${T}_{0}$とすると、
$$
hit\_rate = \frac {T_{1} + T_{0}}{N}
$$
で求めることができる。

### 予測精度(predict precision)
 - $a$: $C$に属するとモデルが予測した評価データのうち、実際に$C$に属しているものの数
 - $b$: $C$に属するとモデルが予測した評価データのうち、$C$に属していないものの数

$$
precision = \frac {a}{a+b}
$$

---  
# 検定  
## モデルの有意性検定 - 尤度比検定  
 - 説明変数が一つしかない場合の有意性検定  

 - $r$個の説明変数の中で$q$個の説明変数の有意性検定

検定仮説
<div style="text-align:center">
帰無仮説 $H_{0}: {\beta}_{r-q+1} = {\beta}_{r-q+2} = \cdots ={\beta}_{r}=0$  
対立仮説 $H_{1}: H_{0}ではない.どれかの変数の係数が0ではない$
</div>

検定統計量
$$
G=-2\log { \left\{ \frac { q個の変数を除いたモデルの尤度 }{ r個の変数全部を含めたモデルの尤度 }  \right\}  } \\ \quad =\quad 2L\left( { \beta  }_{ 0 },{ \beta  }_{ 1 },\cdots ,{ \beta  }_{ r } \right) -2L\left( { \beta  }_{ 0 },{ \beta  }_{ 1 },\cdots ,{ \beta  }_{ r-q },0,\cdots ,0 \right) 
$$
これが、帰無仮説$H_{0}$のもとで斬近的に自由度$q$の${\chi}^{2}$分布に従うことを利用して検定する。

 - モデルの有意性検定

モデルの有意性、つまり
<div style="text-align:center">
帰無仮説 $H_{0}: {\beta}_{0} = {\beta}_{r-q+2} = \cdots ={\beta}_{r}=0$  
対立仮説 $H_{1}: H_{0}ではない.どれかの変数の係数が0ではない$
</div>
に関しては同様に統計量Gが
$$
G=-2\log { \left\{ \frac { r個の変数を除いたモデルの尤度 }{ r個の変数全部を含めたモデルの尤度 }  \right\}  } \\ \quad =\quad 2L\left( { \beta  }_{ 0 },{ \beta  }_{ 1 },\cdots ,{ \beta  }_{ r } \right) -2L\left( 0,0,\cdots ,0 \right) 
$$
自由度$r$の${\chi}^{2}$分布することから検定できる。

## モデルの有意性検定 - Wald検定
説明変数群の有意性検定でない場合のときは、Wald検定を使う。

次式のある説明変数を検定する。
$$
logit\left\{p\left(x\right)\right\} = {\beta}_{0}+{\beta}_{1}{x}_{1}+{\beta}_{2}{x}_{2}+\cdots+{\beta}_{r}{x}_{r}
$$
検定仮説
<div style="text-align:center">
帰無仮説 $H_{0}: {\beta}_{i} = 0$  
対立仮説 $H_{1}: {\beta}_{i} \neq 0$
</div>
検定統計量
$$
Z = \frac {\hat {{\beta}_{i}}}{s.e.\left(\hat {{\beta}_{i}}\right)} \sim 標準正規分布　N\left(0,1\right)
$$
または、その２乗の値である
$$
{ \chi  }^{ 2 }={ \left( \frac { \hat { { \beta  }_{ i } }  }{ s.e.\left( \hat { { \beta  }_{ i } }  \right)  }  \right)  }^{ 2 }\sim 自由度１の{ \chi  }^{ 2 }分布
$$
を利用する。


## 変数選択


## AIC基準
